In [28]:
import pandas as pd
DATASET_COLUMNS  = ["sentiment", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
dataset = pd.read_csv('C:/Users/IDEH/Desktop/Sentimental/Data/Sentiment140.csv',
                      encoding=DATASET_ENCODING , names=DATASET_COLUMNS)

In [29]:
# sentiment_map = {0: 0, 4: 1}  # 0: negative, 1: positive
# dataset.loc[:, 'sentiment'] = dataset['sentiment'].map(sentiment_map)


In [30]:
# Sample the dataset to reduce size
dataset = dataset.sample(frac=0.001, random_state=0)  # Use 10% of the data
dataset.to_csv("160ksentimentdata2.csv")
dataset

,sentiment,ids,date,flag,user,text
557138,0,2204444171,Wed Jun 17 02:14:00 PDT 2009,NO_QUERY,einmensch,wants to compete! i want hard competition! i w...
349381,0,2017152437,Wed Jun 03 07:56:34 PDT 2009,NO_QUERY,keithmorrison,It seems we are stuck on the ground in Amarill...
182051,0,1967043408,Fri May 29 18:52:13 PDT 2009,NO_QUERY,PunkieDory,where the f are my pinking shears? rarararrrar...
571236,0,2208721054,Wed Jun 17 09:32:48 PDT 2009,NO_QUERY,DYkEY_tYPE,0ff t0 tHE MEEtiN.. i HAtE WhEN PPl V0lUNtEER...
1339637,4,2018731586,Wed Jun 03 10:25:27 PDT 2009,NO_QUERY,BlueSmartiies,@ reply me pls
...,...,...,...,...,...,...
92222,0,1759931251,Sun May 10 20:16:36 PDT 2009,NO_QUERY,SteveCoulson,@amandagravel aww you get &quot;saddest tweet ...
1488542,4,2068514157,Sun Jun 07 14:12:42 PDT 2009,NO_QUERY,Genelly1238,"@mitchelmusso ,,hey youre ausome. please give ..."
657354,0,2240992680,Fri Jun 19 10:24:42 PDT 2009,NO_QUERY,firebuilt,"Laptop got a virus, kept crashing, had to rest..."
1368785,4,2050677502,Fri Jun 05 19:05:57 PDT 2009,NO_QUERY,CICELYCORINNE,@oOoshecutee hey hey now!!


In [31]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 557138 to 1246690
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  1600 non-null   int64 
 1   ids        1600 non-null   int64 
 2   date       1600 non-null   object
 3   flag       1600 non-null   object
 4   user       1600 non-null   object
 5   text       1600 non-null   object
dtypes: int64(2), object(4)
memory usage: 87.5+ KB


preprocessing

In [5]:
import re
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import time

# Define your custom preprocessing function with tqdm and time
def preprocess(textdata):
    processedText = []
    wordLemm = WordNetLemmatizer()
    urlPattern = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    userPattern = '@[^\s]+'
    alphaPattern = "[^a-zA-Z0-9]"
    sequencePattern = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"
    
    start_time = time.time()
    
    for tweet in tqdm(textdata, desc="Processing tweets", unit="tweet"):
        tweet = tweet.lower()
        tweet = re.sub(urlPattern, ' URL', tweet)
        tweet = re.sub(userPattern, ' USER', tweet)
        tweet = re.sub(alphaPattern, " ", tweet)
        tweet = re.sub(sequencePattern, seqReplacePattern, tweet)
        tweetwords = ''
        for word in tweet.split():
            word = wordLemm.lemmatize(word)
            tweetwords += (word + ' ')
        processedText.append(tweetwords)
    
    end_time = time.time()
    print(f"Preprocessing completed in {end_time - start_time:.2f} seconds")
    
    return processedText

# Apply the preprocessing function to the text data
processed_text = preprocess(dataset['text'].values)


Processing tweets: 100%|█████████████████████████████████████████████████| 160000/160000 [00:11<00:00, 13781.56tweet/s]

Preprocessing completed in 11.62 seconds


 Feature Extraction : TfidfVectorizer

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
import time
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000, stop_words='english', ngram_range=(1, 2), min_df=5, max_df=0.8)

# Measure the time taken for vectorization
start_time = time.time()

X_tfidf = tfidf_vectorizer.fit_transform(tqdm(processed_text, desc="Vectorizing text data", unit="document"))

end_time = time.time()
print(f"Vectorization completed in {end_time - start_time:.2f} seconds")

# Split the data into training and testing sets
start_time = time.time()

X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, dataset['sentiment'].values, test_size=0.2, random_state=0)

end_time = time.time()
print(f"Data splitting completed in {end_time - start_time:.2f} seconds")


Vectorizing text data: 100%|██████████████████████████████████████████| 160000/160000 [00:02<00:00, 57650.72document/s]


Vectorization completed in 5.08 seconds
Data splitting completed in 0.03 seconds


In [7]:
X_train_dense = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
X_test_dense = pd.DataFrame(X_test_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

MemoryError: Unable to allocate 9.54 GiB for an array with shape (128000, 10000) and data type float64

model

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)

In [ ]:
models, predictions = clf.fit(X_train_tfidf, X_test_tfidf, y_train, y_test)


In [ ]:
models

In [8]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np
import re
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier

# Load the dataset
DATASET_COLUMNS  = ["sentiment", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"
dataset = pd.read_csv('C:/Users/IDEH/Desktop/Sentimental/Data/Sentiment140.csv',
                      encoding=DATASET_ENCODING, names=DATASET_COLUMNS)

# Map sentiment values
sentiment_map = {0: 0, 4: 1}  # 0: negative, 1: positive
dataset.loc[:, 'sentiment'] = dataset['sentiment'].map(sentiment_map)

# Sample the dataset to reduce size
dataset = dataset.sample(frac=0.1, random_state=0)  # Use 10% of the data

# Preprocessing function
def preprocess(textdata):
    processedText = []
    wordLemm = WordNetLemmatizer()
    urlPattern = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    userPattern = '@[^\s]+'
    alphaPattern = "[^a-zA-Z0-9]"
    sequencePattern = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"
    
    start_time = time.time()
    
    for tweet in tqdm(textdata, desc="Processing tweets", unit="tweet"):
        tweet = tweet.lower()
        tweet = re.sub(urlPattern, ' URL', tweet)
        tweet = re.sub(userPattern, ' USER', tweet)
        tweet = re.sub(alphaPattern, " ", tweet)
        tweet = re.sub(sequencePattern, seqReplacePattern, tweet)
        tweetwords = ''
        for word in tweet.split():
            word = wordLemm.lemmatize(word)
            tweetwords += (word + ' ')
        processedText.append(tweetwords)
    
    end_time = time.time()
    print(f"Preprocessing completed in {end_time - start_time:.2f} seconds")
    
    return processedText

# Apply the preprocessing function to the text data
processed_text = preprocess(dataset['text'].values)

# Initialize the TfidfVectorizer with reduced features
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1, 2), min_df=5, max_df=0.8)

# Measure the time taken for vectorization
start_time = time.time()

X_tfidf = tfidf_vectorizer.fit_transform(tqdm(processed_text, desc="Vectorizing text data", unit="document"))

end_time = time.time()
print(f"Vectorization completed in {end_time - start_time:.2f} seconds")

# Split the data into training and testing sets
start_time = time.time()

X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, dataset['sentiment'].values, test_size=0.2, random_state=0)

end_time = time.time()
print(f"Data splitting completed in {end_time - start_time:.2f} seconds")

# Initialize and fit the LazyClassifier with sparse matrices
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train_tfidf, X_test_tfidf, y_train, y_test)

# Display the models
print(models)


Processing tweets: 100%|█████████████████████████████████████████████████| 160000/160000 [00:10<00:00, 14872.56tweet/s]


Preprocessing completed in 10.76 seconds


Vectorizing text data: 100%|██████████████████████████████████████████| 160000/160000 [00:02<00:00, 59487.37document/s]


Vectorization completed in 4.94 seconds
Data splitting completed in 0.02 seconds


AttributeError: select_dtypes not found